<a href="https://colab.research.google.com/github/yaswanth-n-s-n/Advanced-Deep-Learning-with-Keras/blob/master/VRE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-epvm2zwq
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-epvm2zwq
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101065 sha256=9aa2ca88f4810a54e19681eba0c02d059a70e512920bcb8058de8060c7344e64
  Stored in directory: /tmp/pip-ephem-wheel-cache-g37poi5a/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [2]:
# import matplotlib.pyplot as plt
#from __future__ import division
#from __future__ import print_function

from keras.layers import Activation, Dense, Input
from keras.layers import Conv2D, Flatten
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.optimizers import RMSprop
from keras.models import Model
from keras.models import load_model

from keras.layers.merge import concatenate

from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
# from /content/keras-contrib/keras_contrib/layers/normalization/instancenormalization.py import InstanceNormalization

Using TensorFlow backend.


In [0]:
import cv2
import os
import numpy as np
vidcap1 = cv2.VideoCapture('1080.mkv')
vidcap3 = cv2.VideoCapture('1080.mkv')
vidcap2 = cv2.VideoCapture('720.mkv')

In [0]:
from keras.applications.vgg19 import VGG19
import keras.backend as K

def vgg_loss(y_true, y_pred):

    vgg19 = VGG19(include_top=False, weights='imagenet', input_shape=[1080, 1920, 3])
    vgg19.trainable = False
    for l in vgg19.layers:
        l.trainable = False
    loss_model = Model(inputs=vgg19.input, outputs=vgg19.get_layer('block5_conv4').output)
    loss_model.trainable = False
    return K.mean(K.square(loss_model(y_true) - loss_model(y_pred)))

In [0]:
def encoder_layer(inputs,
                  filters=16,
                  kernel_size=3,
                  strides=1,
                  activation='relu',
                  instance_norm=True):
    conv = Conv2D(filters=filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same')

    x = inputs
    if instance_norm:
        x = InstanceNormalization()(x)
    if activation == 'relu':
        x = Activation('relu')(x)
    else:
        x = LeakyReLU(alpha=0.2)(x)
    x1 = conv(x)
    if instance_norm:
        x = InstanceNormalization()(x1)
    if activation == 'relu':
        x = Activation('relu')(x)
    else:
        x = LeakyReLU(alpha=0.2)(x)
    x2 = conv(x)
    x = concatenate([x1,x2])
    return x

In [0]:

def decoder_layer(inputs,
                  paired_inputs,
                  filters=16,
                  kernel_size=3,
                  strides=1,
                  activation='relu',
                  instance_norm=False):
    conv = Conv2DTranspose(filters=filters,
                           kernel_size=kernel_size,
                           strides=strides,
                           padding='same')

    x = inputs
    #if instance_norm:
       # x = InstanceNormalization()(x)
    if activation == 'relu':
        x = Activation('relu')(x)
    else:
        x = LeakyReLU(alpha=0.2)(x)
    x = conv(x)
    x = concatenate([x, paired_inputs])
    return x

In [0]:
def build_generator(input_shape,
                    output_shape,
                    kernel_size=3,
                    name=None):
    inputs_hr = Input(shape=output_shape)
    inputs_lr = Input(shape=input_shape)
    channels = int(output_shape[-1])
    e1 = encoder_layer(inputs_hr,
                       32, strides = 3,
                       kernel_size=kernel_size,
                       activation='leaky_relu')
    e2 = encoder_layer(e1,
                       64,
                       activation='leaky_relu',
                       kernel_size=kernel_size,
                      strides = 1)
    e3 = encoder_layer(e2,
                       128,
                       activation='leaky_relu',strides = 1,
                       kernel_size=kernel_size)
    e4 = encoder_layer(e3,
                       3,
                       activation='leaky_relu',strides = 1,
                       kernel_size=kernel_size)
    e5 = encoder_layer(inputs_lr,3,activation='leaky_relu',strides = 2,
                       kernel_size=kernel_size)
    inp = concatenate([e5, e4])
    d1 = decoder_layer(inp,
                       e3,
                       128,
                       kernel_size=kernel_size)
    d2 = decoder_layer(d1,
                       e2,
                       64,
                       kernel_size=kernel_size)
    d3 = decoder_layer(d2,
                       e1,
                       32,
                       kernel_size=kernel_size)
    d3 = Conv2DTranspose(channels,
                              kernel_size=kernel_size,
                              strides=3,
                              padding='same')(d3)
    outputs = Conv2DTranspose(channels,
                              kernel_size=kernel_size,
                              strides=1,
                              padding='same')(d3)
    generator = Model((inputs_hr,inputs_lr), outputs, name=name)

    return generator

In [0]:
def build_discriminator(input_shape,
                        kernel_size=3,
                        patchgan=True,
                        name=None):

    inputs = Input(shape=input_shape)
    x = encoder_layer(inputs,
                      16,strides=2,
                      kernel_size=kernel_size,
                      activation='leaky_relu',
                      instance_norm=False)
    x = encoder_layer(x,
                      32,strides=2,
                      kernel_size=kernel_size,
                      activation='leaky_relu',
                      instance_norm=False)
    x = encoder_layer(x,
                      64,
                      kernel_size=kernel_size,
                      activation='leaky_relu',
                      strides=2,
                      instance_norm=False)
    x = encoder_layer(x,
                      128,
                      kernel_size=kernel_size,
                      strides=2,
                      activation='leaky_relu',
                      instance_norm=False)

    if patchgan:
        x = LeakyReLU(alpha=0.2)(x)
        x = Conv2D(1,kernel_size=kernel_size,
                         strides=2,
                         padding='same')(x)
    else:
        x = Flatten()(x)
        x = Dense(1)(x)
    outputs = Activation('sigmoid')(x)


    discriminator = Model(inputs, outputs, name=name)

    return discriminator

In [0]:
 def build_cyclegan(dataset_hr, dataset_movement, dataset_hr1):
#     source_shape, target_shape = shapes
    mov_shape = (720, 1280, 3)
    hr_shape = (1080, 1920 ,3)
    lr = 2e-4
    decay = 6e-8
    kernel_size = 3
    batch_size = 1
    gen = build_generator(mov_shape, hr_shape, kernel_size = kernel_size)
#     gen = load_model('gen_1080_vgg_1_1_0_1.h5', custom_objects={"InstanceNormalization": InstanceNormalization})
    gen.summary()
    disc = build_discriminator(hr_shape, kernel_size = kernel_size)
    disc.summary()
    optimizer = RMSprop(lr=0.01*lr, decay = 0.01*decay)
    disc.compile(loss = 'mse', optimizer = optimizer, metrics = ['accuracy'])
    disc.trainable = False
    mov_input = Input(shape = mov_shape)
    hr_input = Input(shape = hr_shape)
    hr_gen = gen([hr_input, mov_input])
    loss = ['mae',vgg_loss, 'mse']
    loss_weights = [0.4, 3, 10]
    inputs = [hr_input, mov_input]
    outputs = [gen([hr_input, mov_input]), gen([hr_input, mov_input]), disc(gen([hr_input, mov_input]))]
    adv = Model(inputs, outputs)
    optimizer = RMSprop(lr = lr, decay = decay)
    adv.compile(loss = loss, loss_weights = loss_weights, optimizer = optimizer, metrics = ['accuracy'])
    adv.summary()
    models = (gen, disc, adv)
    params = (batch_size, int(15000/batch_size), None)
    train_cycle(models, params, dataset_hr, dataset_movement, dataset_hr1)

In [0]:
def train_cycle(models, params, vidcap1, vidcap2, vidcap3):
    gen, disc, adv = models
    batch_size, train_steps, model_name = params
#     mov_data, hr_data, lr_data = data

#     titles, dirs = test_params

    save_interval = 1000
#     target_size = (1280,720,3)
#     source_size = (1280,720,3)
    patch = 15
    if patch > 1:
        d_patch = (34, 60, 1)
        valid = np.ones((batch_size,) + d_patch)
        fake = np.zeros((batch_size,) + d_patch)
    else:
        valid = np.ones([batch_size, 1])
        fake = np.zeros([batch_size, 1])

    valid_fake = np.concatenate((valid, fake))
#     start_time = datetime.now()
    count = 0
    cout=0
    for j in range(20):
        count = 0
        print(j)
        success = True
        for i in range(train_steps):
            vidcap2.read()
            vidcap3.read()
            imagesh1 = []
            imagesL1 = []
            imagesh2 = []
            count = 0
            while success and count<batch_size:
                success,image1 = vidcap1.read()
                imagesh1.append(image1)
                count += 1 
#                 imagesL = []
                success,image2 = vidcap2.read()
                imagesL1.append(image2)
#                 image_ref = image2
#                 count += 1
#                 imagesL = []
                success,image3 = vidcap3.read()
                imagesh2.append(image3)
#                 count += 1
            hr_images = np.array(imagesh1)
            mov_images = np.array(imagesL1)
            hr1_images = np.array(imagesh2)
    #         hr_images = np.asarray(dataset_hr.next().astype(int)) #high resolution image dataset
#             hr_images = dataset_hr[count:count+batch_size]
#             mov_images = dataset_mov[count:count+batch_size]
#             hr1_images = dataset_hr1[count:count+batch_size]
#             count+=batch_size
#             hr_images = normalize(hr_images)
#             hr1_images = normalize(hr1_images)
#     #         mov_images = np.asarray(dataset_movement.next().astype(int)) #movement from one frame to other
#             mov_images = normalize(mov_images)
    #         hr1_images = np.asarray(dataset_hr1.next().astype(int))
    #         cv2_imshow(hr_images)
    #         print(type(hr_images))
            gen_images = gen.predict([hr_images, mov_images]) #generated high resolution images
#             if(i==20): print(gen_images)
    #         print(gen_images.shape())
    #         hr_images = np.array(hr_images.tolist())
            gen_images = np.asarray(gen_images)
            
#             gen_images = denormalize(gen_images)
#             if( i == 100):
#                print(np.shape(gen_images))
#                cv2_imshow(denormalize(gen_images[0]))
#                cv2_imshow(denormalize(hr_images[0]))
#                cv2_imshow(denormalize(hr1_images[0]))
#                cv2_imshow(hr_images[1])
#                cv2_imshow(gen_images[1])
#                cv2_imshow(hr1_images[1])
    #         print(type(hr_images))
    #         print(type(gen_images))
    #         print(len(hr_images))
            x = np.concatenate((hr_images, gen_images)) #concatenating real and fake images to train discriminator
    #         x = np.concatenate[hr_images, fake_images])
            y = np.ones([2*batch_size,1]) #outputs
            y[batch_size:, :]=0.0 # 1 for real and 0 for fake
            loss, acc = disc.train_on_batch(x,valid_fake) #training the discriminator
            log2 = "%d: [discriminator loss: %f, acc: %f]" % (i, loss, acc)
            print(log2)
            y1 = np.ones([batch_size, 1])
            y=[hr1_images,hr1_images, valid] #output for adversial network
            x= [hr_images, mov_images] #input for adversial network
            log = adv.train_on_batch(x,y) #training adversial network
            log1 = "%s: [%f %f %f %f %f]" % (i, log[0], log[1], log[2], log[3], log[4])
            print(log)
              
            if (i+1) % save_interval == 0:
                cout = cout+1
                if (i+1) == train_steps:
                    show = True
                else:
                    show = False
    #             plot_images(generator, noise_input, show = show, steps = (i+1), model_name = model_name)
                gen.save(("gen_1080_vgg_1_1_1_%d_%d.h5")% (j, cout))

In [0]:
build_cyclegan(vidcap1, vidcap2, vidcap3)

Model: "model_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_37 (InputLayer)           (None, 1080, 1920, 3 0                                            
__________________________________________________________________________________________________
instance_normalization_31 (Inst (None, 1080, 1920, 3 2           input_37[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_61 (LeakyReLU)      (None, 1080, 1920, 3 0           instance_normalization_31[0][0]  
__________________________________________________________________________________________________
conv2d_61 (Conv2D)              (None, 360, 640, 32) 896         leaky_re_lu_61[0][0]             
___________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0: [discriminator loss: 0.366012, acc: 0.263480]
[52.004093, 57.826324, 8.792949, 0.24947175, 0.28016928, 0.28016928, 0.6661765]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1: [discriminator loss: 0.350697, acc: 0.275980]
[51.764748, 57.73823, 8.727851, 0.24859, 0.30205005, 0.30205005, 0.6696078]
2: [discriminator loss: 0.339469, acc: 0.312500]
[52.13977, 57.63624, 8.870626, 0.2473397, 0.32498264, 0.32498264, 0.62892157]
3: [discriminator loss: 0.330820, acc: 0.360539]
[51.143543, 57.158497, 8.606979, 0.24592063, 0.3580242, 0.3580242, 0.5598039]
4: [discriminator loss: 0.322139, acc: 0.362500]
[51.392097, 56.537987, 8.779815, 0.24374636, 0.33400753, 0.33400753, 0.5955882]
5: [discriminator loss: 0.316079, acc: 0.411029]
[50.06701, 56.138752, 8.400522, 0.24099441, 0.36267507, 0.36267507, 0.525]
6: [discriminator loss: 0.314758, acc: 0.401961]
[49.877197, 56.020966, 8.363044, 0.23796797, 0.34268278, 0.34268278, 0.56764704]
7: [discriminator loss: 0.308800, acc: 0.435784]
[49.884647, 56.20847, 8.348532, 0.23556632, 0.3636526, 0.3636526, 0.5269608]
8: [discriminator loss: 0.305984, acc: 0.444853]
[49.734215, 65.156845, 7.116713, 0.23213358, 0.347432, 0.347432